Issue: Can argumentative elements be better identified if they are "effective"?

In [1]:
import itertools
import pandas as pd
import nltk
import experiment_util

Load gold-labeled data from the kaggle-competition as DataFrame and rename the columns such that existing code can be reused.  
Also load predictions from the "Don't Drop the Topic"-experiments (https://aclanthology.org/2022.bea-1.17/) and rename the columns.

In [2]:
renaming_gold = {'essay_id': 'id', 'prediction_string': 'predictionstring'}
renaming_pred = {'essay_id': 'id', 'prediction_string': 'predictionstring', 'discourse_type': 'class'}

gold_df = pd.read_csv("merged.csv")
gold_df.rename(columns=renaming_gold, inplace=True)
prediction_df = pd.read_csv("train1test2_prediction.csv")
prediction_df.rename(columns=renaming_pred, inplace=True)

Append an extra column with the effectiveness to the gold_df.  
These additional gold-labels were gathered from a new kaggle-competition.

In [3]:
clusters_df = pd.read_csv('clusters_effectivness.csv').set_index('id')
gold_df = gold_df.join(clusters_df, on='id')

Add columns based on the discourse_text containing:
- only function words kept (content words replaced by their POS-tags),
- bigrams of above
- trigrams of above

It is assumed that function words can be identified by the closed-class POS-tags below.

In [4]:
closed_class_pos_tags = ["Cd", "CC", "DT", "EX", "IN", "LS", "MD", "PDT", "POS",
                         "PRP", "PRP$", "RP", "TO", "UH", "WDT", "WP", "WP$", "WRB"]
gold_df['dt_tokenized'] = [nltk.tokenize.word_tokenize(dt) for dt in gold_df['discourse_text']]
tagged_tokens_dts = [nltk.pos_tag(dt) for dt in gold_df['dt_tokenized']]
dts_only_function_words = []
dts_ofw_bigrams = []
dts_ofw_trigrams = []
for tagged_tokens in tagged_tokens_dts:
    dt_only_function_words = [(tagged_token[0] if tagged_token[1] in closed_class_pos_tags else tagged_token[1]) for tagged_token in tagged_tokens]
    dts_only_function_words.append(dt_only_function_words)
    dts_ofw_bigrams.append(list(nltk.bigrams(dt_only_function_words)))
    dts_ofw_trigrams.append(list(nltk.trigrams(dt_only_function_words)))

gold_df['dt_only_function_words'] = dts_only_function_words
gold_df['dt_ofw_bigrams'] = dts_ofw_bigrams
gold_df['dt_ofw_trigrams'] = dts_ofw_trigrams
gold_df

,discourse_id,id,discourse_text,discourse_type,discourse_effectiveness,predictionstring,cluster,dt_tokenized,dt_only_function_words,dt_ofw_bigrams,dt_ofw_trigrams
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...,1,"[Hi, ,, i, 'm, Isaac, ,, i, 'm, going, to, be,...","[NNP, ,, NN, VBP, NNP, ,, NN, VBP, VBG, to, VB...","[(NNP, ,), (,, NN), (NN, VBP), (VBP, NNP), (NN...","[(NNP, ,, NN), (,, NN, VBP), (NN, VBP, NNP), (..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 8...,1,"[On, my, perspective, ,, I, think, that, the, ...","[On, my, NN, ,, I, VBP, that, the, NN, VBZ, a,...","[(On, my), (my, NN), (NN, ,), (,, I), (I, VBP)...","[(On, my, NN), (my, NN, ,), (NN, ,, I), (,, I,..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,108 109 110 111 112 113 114 115 116 117 118 11...,1,"[I, think, that, the, face, is, a, natural, la...","[I, VBP, that, the, NN, VBZ, a, JJ, NN, becaus...","[(I, VBP), (VBP, that), (that, the), (the, NN)...","[(I, VBP, that), (VBP, that, the), (that, the,..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,129 130 131 132 133 134 135 136 137 138 139 14...,1,"[If, life, was, on, Mars, ,, we, would, know, ...","[If, NN, VBD, on, NNP, ,, we, would, VB, by, R...","[(If, NN), (NN, VBD), (VBD, on), (on, NNP), (N...","[(If, NN, VBD), (NN, VBD, on), (VBD, on, NNP),..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,201 202 203 204 205 206 207 208 209 210 211 21...,1,"[People, thought, that, the, face, was, formed...","[NNS, VBD, that, the, NN, VBD, VBN, by, NNS, b...","[(NNS, VBD), (VBD, that), (that, the), (the, N...","[(NNS, VBD, that), (VBD, that, the), (that, th..."
...,...,...,...,...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,Claim,Adequate,14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 2...,5,"[For, many, people, they, do, n't, like, only,...","[For, JJ, NNS, they, VBP, RB, like, RB, VBG, C...","[(For, JJ), (JJ, NNS), (NNS, they), (they, VBP...","[(For, JJ, NNS), (JJ, NNS, they), (NNS, they, ..."
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,Claim,Adequate,35 36 37 38 39 40 41 42 43,5,"[also, people, have, different, views, and, op...","[RB, NNS, VBP, JJ, NNS, and, NNS, on, NNS, .]","[(RB, NNS), (NNS, VBP), (VBP, JJ), (JJ, NNS), ...","[(RB, NNS, VBP), (NNS, VBP, JJ), (VBP, JJ, NNS..."
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,Position,Adequate,44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 5...,5,"[Advice, is, something, that, can, impact, a, ...","[NNP, VBZ, NN, that, can, VB, a, NNS, NN, in, ...","[(NNP, VBZ), (VBZ, NN), (NN, that), (that, can...","[(NNP, VBZ, NN), (VBZ, NN, that), (NN, that, c..."
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,Evidence,Ineffective,80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 9...,5,"[someone, can, use, everything, that, many, pe...","[NN, can, VB, NN, that, JJ, NNS, VBD, and, VB,...","[(NN, can), (can, VB), (VB, NN), (NN, that), (...","[(NN, can, VB), (can, VB, NN), (VB, NN, that),..."


split up gold_df by effectiveness

In [5]:
gold_df_effective = gold_df[gold_df.discourse_effectiveness == 'Effective']
gold_df_effective.to_csv('gold_effective.csv')
gold_df_adequate = gold_df[gold_df.discourse_effectiveness == 'Adequate']
gold_df_adequate.to_csv('gold_adequate.csv')
gold_df_ineffective = gold_df[gold_df.discourse_effectiveness == 'Ineffective']
gold_df_ineffective.to_csv('gold_ineffective.csv')

Evaluate the prediction scores for all, effective, adequate and ineffective discourses respectively.  
Notice that **F1** is **not well suited** for comparison **here**.
This is because any unmatched predictions are counted as *false positives*.

In [6]:
experiment_util.model_evaluate(prediction_df, gold_df)[1]

Overall evaluation: 0.5544623497603323


{'Lead': {'TP': 1673,
  'FP': 699,
  'FN': 619,
  'Precision': 0.7299301919720768,
  'Recall': 0.7053119730185498,
  'F1': 0.717409948542024},
 'Position': {'TP': 1965,
  'FP': 1104,
  'FN': 2060,
  'Precision': 0.48819875776397514,
  'Recall': 0.6402737047898338,
  'F1': 0.5539892867211729},
 'Claim': {'TP': 4118,
  'FP': 4255,
  'FN': 7860,
  'Precision': 0.34379696109534147,
  'Recall': 0.49181894183685654,
  'F1': 0.4046975578595646},
 'Evidence': {'TP': 7321,
  'FP': 4649,
  'FN': 4785,
  'Precision': 0.6047414505204031,
  'Recall': 0.6116123642439432,
  'F1': 0.6081575012460542},
 'Counterclaim': {'TP': 836,
  'FP': 931,
  'FN': 938,
  'Precision': 0.47125140924464487,
  'Recall': 0.4731182795698925,
  'F1': 0.4721829991527817},
 'Rebuttal': {'TP': 460,
  'FP': 614,
  'FN': 785,
  'Precision': 0.36947791164658633,
  'Recall': 0.42830540037243947,
  'F1': 0.39672272531263475},
 'Concluding Statement': {'TP': 2553,
  'FP': 1108,
  'FN': 799,
  'Precision': 0.7616348448687351,
  'Re

In [7]:
experiment_util.model_evaluate(prediction_df.copy(), gold_df_ineffective.copy())[1]

Overall evaluation: 0.1195244716633719


{'Rebuttal': {'TP': 60,
  'FP': 1014,
  'FN': 152,
  'Precision': 0.2830188679245283,
  'Recall': 0.055865921787709494,
  'F1': 0.09331259720062209},
 'Evidence': {'TP': 1610,
  'FP': 10356,
  'FN': 1547,
  'Precision': 0.5099778270509978,
  'Recall': 0.13454788567608222,
  'F1': 0.2129207167889969},
 'Claim': {'TP': 457,
  'FP': 7878,
  'FN': 1019,
  'Precision': 0.30962059620596205,
  'Recall': 0.05482903419316137,
  'F1': 0.09316073794720212},
 'Counterclaim': {'TP': 70,
  'FP': 1697,
  'FN': 136,
  'Precision': 0.33980582524271846,
  'Recall': 0.039615166949632144,
  'F1': 0.07095793208312215},
 'Position': {'TP': 140,
  'FP': 2929,
  'FN': 331,
  'Precision': 0.29723991507430997,
  'Recall': 0.045617464972303685,
  'F1': 0.07909604519774012},
 'Lead': {'TP': 186,
  'FP': 2186,
  'FN': 179,
  'Precision': 0.5095890410958904,
  'Recall': 0.07841483979763912,
  'F1': 0.1359152356594812},
 'Concluding Statement': {'TP': 321,
  'FP': 3340,
  'FN': 261,
  'Precision': 0.5515463917525774

In [8]:
experiment_util.model_evaluate(prediction_df.copy(), gold_df_adequate.copy())[1]

Overall evaluation: 0.4020783330457223


{'Lead': {'TP': 886,
  'FP': 1486,
  'FN': 359,
  'Precision': 0.7116465863453815,
  'Recall': 0.37352445193929174,
  'F1': 0.489908764169201},
 'Position': {'TP': 1394,
  'FP': 1675,
  'FN': 1391,
  'Precision': 0.5005385996409336,
  'Recall': 0.4542196155099381,
  'F1': 0.4762555517594807},
 'Claim': {'TP': 2271,
  'FP': 6083,
  'FN': 4827,
  'Precision': 0.31994928148774304,
  'Recall': 0.27184582236054583,
  'F1': 0.29394253171110535},
 'Evidence': {'TP': 3457,
  'FP': 8508,
  'FN': 2608,
  'Precision': 0.5699917559769168,
  'Recall': 0.28892603426661095,
  'F1': 0.3834719911259013},
 'Counterclaim': {'TP': 497,
  'FP': 1270,
  'FN': 654,
  'Precision': 0.4317984361424848,
  'Recall': 0.2812676853423882,
  'F1': 0.3406442769019877},
 'Concluding Statement': {'TP': 1554,
  'FP': 2107,
  'FN': 392,
  'Precision': 0.7985611510791367,
  'Recall': 0.42447418738049714,
  'F1': 0.5543071161048689},
 'Rebuttal': {'TP': 244,
  'FP': 830,
  'FN': 450,
  'Precision': 0.3515850144092219,
  'Re

In [9]:
experiment_util.model_evaluate(prediction_df.copy(), gold_df_effective.copy())[1]

Overall evaluation: 0.275240039779121


{'Lead': {'TP': 601,
  'FP': 1771,
  'FN': 83,
  'Precision': 0.8786549707602339,
  'Recall': 0.2533726812816189,
  'F1': 0.3933246073298429},
 'Position': {'TP': 431,
  'FP': 2638,
  'FN': 340,
  'Precision': 0.5590142671854734,
  'Recall': 0.14043662430759205,
  'F1': 0.22447916666666667},
 'Evidence': {'TP': 2254,
  'FP': 9711,
  'FN': 632,
  'Precision': 0.781011781011781,
  'Recall': 0.18838278311742582,
  'F1': 0.30354858258703116},
 'Claim': {'TP': 1390,
  'FP': 6958,
  'FN': 2016,
  'Precision': 0.40810334703464474,
  'Recall': 0.16650694777192143,
  'F1': 0.236515228858261},
 'Concluding Statement': {'TP': 678,
  'FP': 2983,
  'FN': 148,
  'Precision': 0.8208232445520581,
  'Recall': 0.18519530183010108,
  'F1': 0.3022063739692445},
 'Counterclaim': {'TP': 269,
  'FP': 1498,
  'FN': 150,
  'Precision': 0.6420047732696897,
  'Recall': 0.1522354272778721,
  'F1': 0.24611161939615736},
 'Rebuttal': {'TP': 156,
  'FP': 918,
  'FN': 185,
  'Precision': 0.4574780058651026,
  'Recall

Define how to measure the overlap between two arguments.  
`calc_overlap_mtc` is a variation of `experiment_util.calc_overlap`, but it considers multiple occurrences of a token.

In [10]:
def calc_overlap_mtc(a: list, b):
    # Length of each and intersection
    try:
        len_a = len(a)
        len_b = len(b)
        inter = 0
        for tok in (set(a) & set(b)):
            inter += min(a.count(tok), b.count(tok))
        overlap_1 = inter / len_a
        overlap_2 = inter / len_b
        return overlap_1, overlap_2
    except:  # at least one of the input is NaN
        return 0, 0

In [11]:
# similarity-function between two arguments
def argument_similarity(arg1, arg2):
    overlap = calc_overlap_mtc(arg1, arg2)
    return (overlap[0] + overlap[1]) / 2

In [12]:
def mean_arg_similarity_per_selection(gold_df, dt_column, selector = 'cluster', scnd_gold_df = None):
    mean_similarities = dict()
    overall_sum_similarity = 0
    overall_comb_cnt = 0
    for selection in list(set(gold_df[selector].values)):    # for each cluster/ label
        gold_df_selection = gold_df[gold_df[selector] == selection]
        dtext_tokens = gold_df_selection[dt_column]

        if scnd_gold_df is not None:
            gold_df_selection2 = scnd_gold_df[scnd_gold_df[selector] == selection]
            dtext_tokens2 = gold_df_selection2[dt_column]
            dt_combinations = list(itertools.product(dtext_tokens, dtext_tokens2))
        else:
            dt_combinations = list(itertools.combinations(dtext_tokens, 2))

        mean_similarities[selection] = 0
        for dt_combination in dt_combinations:
            arg_similarity = argument_similarity(*dt_combination)
            mean_similarities[selection] += arg_similarity
            overall_sum_similarity += arg_similarity
        mean_similarities[selection] /= len(dt_combinations)
        overall_comb_cnt += len(dt_combinations)

    print("⌀ =", sum(mean_similarities.values()) / len(mean_similarities.values()))
    print("⌀ w =", overall_sum_similarity / overall_comb_cnt)
    return mean_similarities

In [13]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_tokenized', 'cluster')

⌀ = 0.23102810797794338
⌀ w = 0.23258378300264532


{0: 0.2553660146358815,
 1: 0.23979365127001806,
 2: 0.23427370530567104,
 3: 0.2504519478221547,
 4: 0.22444682104281952,
 5: 0.23153646604271697,
 6: 0.23589694532730193,
 7: 0.20704920948803238,
 8: 0.2465095575256454,
 9: 0.22656748282813344,
 10: 0.21439873550258368,
 11: 0.22344570324985838,
 12: 0.21514813570453029,
 13: 0.2273226452217507,
 14: 0.2332145987020533}

In [14]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'dt_tokenized', 'cluster')

⌀ = 0.1820172560765008
⌀ w = 0.1892141956250927


{0: 0.201171482338953,
 1: 0.1795685315730087,
 2: 0.19311806072397436,
 3: 0.1882006634885314,
 4: 0.18858068785696894,
 5: 0.14888545096092365,
 6: 0.17173026578887585,
 7: 0.20898713086424894,
 8: 0.20162169536842123,
 9: 0.17295999089757833,
 10: 0.16186749012766166,
 11: 0.1949377386349762,
 12: 0.14761373852826537,
 13: 0.1803651180704469,
 14: 0.19065079592467762}

In [15]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_tokenized', 'cluster', gold_df_ineffective)

⌀ = 0.1974087607990022
⌀ w = 0.20289743353317716


{0: 0.21613967248740934,
 1: 0.1951705163060201,
 2: 0.2087695704743159,
 3: 0.20747808233064222,
 4: 0.19990741489530628,
 5: 0.18801407259714523,
 6: 0.195446818385501,
 7: 0.19745948604486424,
 8: 0.21488956926890154,
 9: 0.18077204753004752,
 10: 0.18439039003874186,
 11: 0.19826165515075636,
 12: 0.1747680782505798,
 13: 0.1945511144561894,
 14: 0.20511292376861223}

In [16]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_tokenized', 'discourse_type')

⌀ = 0.20213715086974485
⌀ w = 0.18837435698984514


{'Lead': 0.23009351324160646,
 'Position': 0.16130425353319394,
 'Concluding Statement': 0.2636050353610914,
 'Claim': 0.13666628453010496,
 'Rebuttal': 0.19820470069204202,
 'Counterclaim': 0.17100825527584068,
 'Evidence': 0.2540780134543343}

In [17]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'dt_tokenized', 'discourse_type')

⌀ = 0.14412021868046038
⌀ w = 0.17588530504480662


{'Position': 0.12097737566129627,
 'Lead': 0.14130486807803358,
 'Concluding Statement': 0.1648989357802526,
 'Claim': 0.11401936173397711,
 'Rebuttal': 0.15777091955685596,
 'Counterclaim': 0.1181068246673512,
 'Evidence': 0.19176324528545585}

In [18]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_tokenized', 'discourse_type', gold_df_ineffective)

⌀ = 0.16363737601659833
⌀ w = 0.17987875241835014


{'Lead': 0.1793192500816503,
 'Position': 0.13076976221649683,
 'Concluding Statement': 0.19770325234185243,
 'Claim': 0.1181670817132529,
 'Rebuttal': 0.16702289638231493,
 'Counterclaim': 0.13702686550087342,
 'Evidence': 0.21545252387974764}

In [19]:
discourse_types = gold_df['discourse_type'].unique().tolist()

for cluster in range(0, 15):

    mean_similarities = dict()
    overall_sum_similarity = 0
    overall_comb_cnt = 0

    gold_df_cluster = gold_df[gold_df.cluster == cluster]
    for (dtype1, dtype2) in itertools.combinations_with_replacement(discourse_types, 2):
        dtext_tokens_1 = gold_df_cluster[gold_df_cluster.discourse_type == dtype1]['dt_tokenized']
        dtext_tokens_2 = gold_df_cluster[gold_df_cluster.discourse_type == dtype2]['dt_tokenized']
        dt_combinations = list(itertools.product(dtext_tokens_1, dtext_tokens_2))

        mean_similarities[(dtype1, dtype2)] = 0
        for dt_combination in dt_combinations:
            arg_similarity = argument_similarity(*dt_combination)
            mean_similarities[(dtype1, dtype2)] += arg_similarity
            overall_sum_similarity += arg_similarity
        mean_similarities[(dtype1, dtype2)] /= len(dt_combinations)
        overall_comb_cnt += len(dt_combinations)
    print(cluster)
    print(mean_similarities)


0
{('Lead', 'Lead'): 0.24470288173789742, ('Lead', 'Position'): 0.21749157890860435, ('Lead', 'Claim'): 0.18925147683187, ('Lead', 'Evidence'): 0.23212998524319953, ('Lead', 'Counterclaim'): 0.18907975188209544, ('Lead', 'Rebuttal'): 0.18011763264518701, ('Lead', 'Concluding Statement'): 0.2241714376248291, ('Position', 'Position'): 0.2284260355017516, ('Position', 'Claim'): 0.1697518261895235, ('Position', 'Evidence'): 0.20465038160686835, ('Position', 'Counterclaim'): 0.16854520130270936, ('Position', 'Rebuttal'): 0.15948609527769175, ('Position', 'Concluding Statement'): 0.2138089072851158, ('Claim', 'Claim'): 0.15426536403472763, ('Claim', 'Evidence'): 0.1982848226019764, ('Claim', 'Counterclaim'): 0.15434876677644302, ('Claim', 'Rebuttal'): 0.14547792017173633, ('Claim', 'Concluding Statement'): 0.1900283260147172, ('Evidence', 'Evidence'): 0.2463728224251335, ('Evidence', 'Counterclaim'): 0.20006706398218327, ('Evidence', 'Rebuttal'): 0.19646411495856397, ('Evidence', 'Concluding

In [20]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_bigrams', 'cluster')

⌀ = 0.16416527014005883
⌀ w = 0.17094117521016802


{0: 0.18392811136780976,
 1: 0.1605408740979455,
 2: 0.16113375142434655,
 3: 0.17778151026021477,
 4: 0.16935174658083404,
 5: 0.16586453358954922,
 6: 0.14765600135846235,
 7: 0.1398231446247821,
 8: 0.177607103476657,
 9: 0.17090750367014668,
 10: 0.13491567379418085,
 11: 0.18399385851451278,
 12: 0.16302705066159728,
 13: 0.18338251178073203,
 14: 0.1425656768991113}

In [21]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'dt_ofw_bigrams', 'cluster')

⌀ = 0.11745676373847387
⌀ w = 0.12355085461194991


{0: 0.1293351168780475,
 1: 0.10155260558073946,
 2: 0.1280825493465467,
 3: 0.11957282082096352,
 4: 0.1303521924160099,
 5: 0.0988002711922925,
 6: 0.09337078133379698,
 7: 0.14597912062199075,
 8: 0.12797563259162184,
 9: 0.11644211777815199,
 10: 0.09137684616971158,
 11: 0.16846848780354323,
 12: 0.09800122798140652,
 13: 0.10902751037031147,
 14: 0.10351417519197414}

In [22]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_bigrams', 'cluster', gold_df_ineffective)

⌀ = 0.13437035874900408
⌀ w = 0.13837890274188797


{0: 0.14857562108986686,
 1: 0.11866351698002503,
 2: 0.14117810317564414,
 3: 0.14093416052884694,
 4: 0.1454479701549203,
 5: 0.13318848368607825,
 6: 0.11476411445780829,
 7: 0.13433524846318257,
 8: 0.14530014787337667,
 9: 0.13263309458028136,
 10: 0.11068575884493907,
 11: 0.16849266302503155,
 12: 0.12558810091787326,
 13: 0.13613757420002381,
 14: 0.11963082325716332}

In [23]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_bigrams', 'discourse_type')

⌀ = 0.1497849381211653
⌀ w = 0.14122350941108325


{'Lead': 0.18100876640034497,
 'Position': 0.11037151080537236,
 'Concluding Statement': 0.20811706708487449,
 'Claim': 0.09162937494926329,
 'Rebuttal': 0.13977660271919132,
 'Counterclaim': 0.11214880025377001,
 'Evidence': 0.20544244463534053}

In [24]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'dt_ofw_bigrams', 'discourse_type')

⌀ = 0.09037441176829351
⌀ w = 0.12364402961657248


{'Position': 0.07490630715585997,
 'Lead': 0.10643029656640433,
 'Concluding Statement': 0.096090317884012,
 'Claim': 0.06883751470735829,
 'Rebuttal': 0.07634031608170269,
 'Counterclaim': 0.07173671497067434,
 'Evidence': 0.13827941501204305}

In [25]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_bigrams', 'discourse_type', gold_df_ineffective)

⌀ = 0.11527937144578061
⌀ w = 0.13426632277355582


{'Lead': 0.14215464648646314,
 'Position': 0.08660631388842356,
 'Concluding Statement': 0.14324399874272092,
 'Claim': 0.07685386423849387,
 'Rebuttal': 0.10227382900835409,
 'Counterclaim': 0.087993866975062,
 'Evidence': 0.16782908078094666}

In [26]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_trigrams', 'cluster')

⌀ = 0.03447670043118078
⌀ w = 0.03627878285151256


{0: 0.04367752382634346,
 1: 0.03595057427774131,
 2: 0.03641938310859379,
 3: 0.037976433048350994,
 4: 0.03485854977150004,
 5: 0.03469337767307289,
 6: 0.025938519296751544,
 7: 0.03074529979828981,
 8: 0.038434330203106874,
 9: 0.036717854038123596,
 10: 0.024240060786478224,
 11: 0.04018865782532133,
 12: 0.03288977154466823,
 13: 0.037645916794002525,
 14: 0.026774254475367066}

In [27]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'dt_ofw_trigrams', 'cluster')

⌀ = 0.021205766195827994
⌀ w = 0.024070817074008884


{0: 0.02510541685805953,
 1: 0.016422452783658576,
 2: 0.0260332969546827,
 3: 0.01957929465808649,
 4: 0.02198432390225014,
 5: 0.013612316287964376,
 6: 0.013558486132616608,
 7: 0.03352572027674118,
 8: 0.022725768255345166,
 9: 0.020500449642063052,
 10: 0.014428462447288395,
 11: 0.040645692619043924,
 12: 0.01447471826516854,
 13: 0.01806142179766601,
 14: 0.017428672056785204}

In [28]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_trigrams', 'cluster', gold_df_ineffective)

⌀ = 0.024504644728488977
⌀ w = 0.0271396748304852


{0: 0.028947303614045485,
 1: 0.020748668496878182,
 2: 0.029412154603819515,
 3: 0.026091946885626896,
 4: 0.026459621306056554,
 5: 0.020460923993268876,
 6: 0.01759712728486228,
 7: 0.027617537230521405,
 8: 0.027030218500215586,
 9: 0.023286578891667263,
 10: 0.018243276574215803,
 11: 0.03691843811079213,
 12: 0.02071222470272592,
 13: 0.023433830265604506,
 14: 0.020609820467034176}

In [29]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_trigrams', 'discourse_type')

⌀ = 0.028336852389009657
⌀ w = 0.026073798338078557


{'Lead': 0.035959521726264584,
 'Position': 0.019745377808625886,
 'Concluding Statement': 0.043796019038327604,
 'Claim': 0.01288801364941943,
 'Rebuttal': 0.023795462135842032,
 'Counterclaim': 0.01910484302235876,
 'Evidence': 0.043068729342229294}

In [30]:
mean_arg_similarity_per_selection(gold_df_ineffective, 'dt_ofw_trigrams', 'discourse_type')

⌀ = 0.01388272553561454
⌀ w = 0.021549157677740096


{'Position': 0.010386860794531017,
 'Lead': 0.016989488980225086,
 'Concluding Statement': 0.01383085345634225,
 'Claim': 0.008279201736679788,
 'Rebuttal': 0.011500668925770965,
 'Counterclaim': 0.011087873626499005,
 'Evidence': 0.02510413122925369}

In [31]:
mean_arg_similarity_per_selection(gold_df_effective, 'dt_ofw_trigrams', 'discourse_type', gold_df_ineffective)

⌀ = 0.01899626851337512
⌀ w = 0.02366971592267722


{'Lead': 0.024748938683276888,
 'Position': 0.012375562032803007,
 'Concluding Statement': 0.024777651695416278,
 'Claim': 0.009521542170209686,
 'Rebuttal': 0.016147343247087727,
 'Counterclaim': 0.013412032371183002,
 'Evidence': 0.03199080939364925}